In [1]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import re
import redditutils
import pickle


from string import punctuation

from nltk.corpus import stopwords
from nltk.corpus import words
from nltk.tokenize import word_tokenize
from nltk import SnowballStemmer

from sklearn.feature_extraction.text import CountVectorizer
from sklearn.decomposition import LatentDirichletAllocation
from sklearn.ensemble import RandomForestClassifier
from sklearn.naive_bayes import GaussianNB
from sklearn.model_selection import train_test_split
from sklearn.model_selection import cross_val_score
from sklearn.pipeline import make_pipeline
from sklearn.feature_extraction.text import TfidfVectorizer

from scipy.io import mmwrite, mmread

from gensim import corpora, models, similarities, matutils

from xgboost import XGBClassifier

%matplotlib inline
%load_ext autoreload
%autoreload 2

In [2]:
# English words
words_corpus = set(words.words())
# Stop words
stop = set(stopwords.words('english'))
# English words minus stop words
acceptable_words = words_corpus - stop
analyzer = CountVectorizer().build_analyzer()
stem = SnowballStemmer('english')

def english_corpus(doc, stemmer=stem):
    return [stemmer.stem(w) for w in analyzer(doc) if w in acceptable_words]

cv = CountVectorizer(stop_words='english', 
                     min_df = 2,
                     max_df = .10, 
                     tokenizer=english_corpus,
                     strip_accents='unicode',
                     encoding='utf-8', 
                     ngram_range=(1, 2))

In [9]:
train = pd.read_csv('train.csv', index_col = 0)

/anaconda3/lib/python3.6/site-packages/IPython/core/interactiveshell.py:2785: DtypeWarning: Columns (0) have mixed types. Specify dtype option on import or set low_memory=False.
  interactivity=interactivity, compiler=compiler, result=result)


In [10]:
test = pd.read_csv('test.csv', index_col = 0)

/anaconda3/lib/python3.6/site-packages/IPython/core/interactiveshell.py:2785: DtypeWarning: Columns (0) have mixed types. Specify dtype option on import or set low_memory=False.
  interactivity=interactivity, compiler=compiler, result=result)


In [11]:
test.head()

,author,title,subreddit,score,num_comments,retrieved_on,id,created_utc,selftext,quality
700000,livesNbox,I wonder if anyone sells dandelion seeds,Showerthoughts,3.0,2.0,1.494614e+09,66dags,2017-04-19 21:33:46,NaN,1.0
700001,keepmovingon69,"Before phone and telegram lines, you could pro...",Showerthoughts,3.0,2.0,1.463498e+09,4c0ksq,2016-03-26 09:01:46,NaN,1.0
700002,vorpike,As an Asian idk why we stereotypically pronoun...,Showerthoughts,3.0,2.0,1.473036e+09,4x6gd8,2016-08-11 05:17:10,Some of us can't roll our tongues and pronounc...,1.0
700003,CappyCrunch,When you're at someone else's house the loudes...,Showerthoughts,3.0,2.0,1.492418e+09,5zt2xp,2017-03-16 20:28:14,NaN,1.0
700004,Fart_Kontrol,Why didn't Uncle Phillip buy the Fresh Prince'...,Showerthoughts,3.0,2.0,1.492411e+09,5zevhw,2017-03-14 20:56:38,"I mean, Fresh P is in high school. Seems like...",1.0


In [14]:
train.dropna(inplace=True)

In [15]:
train['quality'] = train['quality'].astype(int)

In [18]:
resp = train['quality']
feat = train['title']

In [21]:
X_train, X_test, y_train, y_test = train_test_split(feat, resp, stratify = resp)

In [24]:
X_train_short = X_train[:50000]
y_train_short = y_train[:50000]

In [22]:
# English words
words_corpus = set(words.words())
# Stop words
stop = set(stopwords.words('english'))
# English words minus stop words
acceptable_words = words_corpus - stop
analyzer = CountVectorizer().build_analyzer()
stem = SnowballStemmer('english')

def english_corpus(doc, stemmer=stem):
    return [stemmer.stem(w) for w in analyzer(doc) if w in acceptable_words]

tfidf = TfidfVectorizer(stop_words = "english",
                        strip_accents = 'ascii',
                        max_df = .10,
                        min_df = 3, 
                        tokenizer = english_corpus,
                        ngram_range=(1, 2))

In [39]:
X_train_short[0]

'We try to figure out what conscioussnes is on a neurological level but this whole concept of consciousness was created by our own brain'

In [37]:
X_tf = tfidf.fit_transform(X_train)

KeyError: 'title'

In [30]:
X_tf

<362697x121310 sparse matrix of type '<class 'numpy.float64'>'
	with 2841219 stored elements in Compressed Sparse Row format>

In [40]:
X_tf2 = cv.fit_transform(X_train_short)

In [ ]:
rfc = RandomForestClassifier()

cross_val_score(rfc, X_tf2.toarray(), y_train_short, cv=3, scoring='roc_auc')

/anaconda3/lib/python3.6/site-packages/sklearn/ensemble/forest.py:248: FutureWarning: The default value of n_estimators will change from 10 in version 0.20 to 100 in 0.22.
  "10 in version 0.20 to 100 in 0.22.", FutureWarning)
/anaconda3/lib/python3.6/site-packages/sklearn/ensemble/forest.py:248: FutureWarning: The default value of n_estimators will change from 10 in version 0.20 to 100 in 0.22.
  "10 in version 0.20 to 100 in 0.22.", FutureWarning)
